In [2]:
import uuid, sys, os
import pandas as pd
import numpy as np
from tqdm import tqdm
import ast
import math
import random

from sklearn import metrics
from scipy import stats
from collections import Counter

os.environ["CUDA_VISIBLE_DEVICES"] = "3"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
import torch
torch.cuda.set_device(0)  # 0 == "first visible" -> actually GPU 2 on the node
print(torch.cuda.get_device_name(0))

import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset, WeightedRandomSampler
import pytorch_lightning as pl
from torch.optim import AdamW

torch.manual_seed(0)

from accelerate import Accelerator
torch.cuda.empty_cache()
import training_utils.partitioning_utils as pat_utils

import warnings
from pydantic._internal._generate_schema import UnsupportedFieldAttributeWarning

Tesla V100-SXM2-32GB


In [3]:
SEED = 0
os.environ["PYTHONHASHSEED"] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

In [4]:
import requests
requests.get("https://api.wandb.ai/status").status_code

404

In [5]:
import wandb
wandb.login(key="f8a6d759fe657b095d56bddbdb4d586dfaebd468")

/work3/s232958/envs/esm_gpu/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/work3/s232958/envs/esm_gpu/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annot

True

In [6]:
os.chdir("/zhome/c9/0/203261/DBL046_PP_osaul/DBL046_PP_osaul/tmp/ona_drafts")
# print(os.getcwd())

print("PyTorch:", torch.__version__)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print("Current location:", os.getcwd())

PyTorch: 2.5.1
Using device: cuda
Current location: /zhome/c9/0/203261/DBL046_PP_osaul/DBL046_PP_osaul/tmp/ona_drafts


In [7]:
# Model parameters
memory_verbose = False
use_wandb = True # Used to track loss in real-time without printing
model_save_steps = 1
train_frac = 1.0
test_frac = 1.0

embedding_dimension = 1280 #| 960 | 1152
number_of_recycles = 2
padding_value = -5000

# batch_size = 20
learning_rate = 2e-5
EPOCHS = 15

In [8]:
def create_key_padding_mask(embeddings, padding_value=-5000, offset=10):
    return (embeddings < (padding_value + offset)).all(dim=-1)

def create_mean_of_non_masked(embeddings, padding_mask):
    # Use masked select and mean to compute the mean of non-masked elements
    # embeddings should be of shape (batch_size, seq_len, features)
    seq_embeddings = []
    for i in range(embeddings.shape[0]): # looping over all batch elements
        non_masked_embeddings = embeddings[i][~padding_mask[i]] # shape [num_real_tokens, features]
        if len(non_masked_embeddings) == 0:
            print("You are masking all positions when creating sequence representation")
            sys.exit(1)
        mean_embedding = non_masked_embeddings.mean(dim=0) # sequence is represented by the single vecotr [1152] [features]
        seq_embeddings.append(mean_embedding)
    return torch.stack(seq_embeddings)

class MiniCLIP_w_transformer_crossattn(pl.LightningModule):

    def __init__(self, padding_value = -5000, embed_dimension=embedding_dimension, num_recycles=2):

        super().__init__()
        self.num_recycles = num_recycles # how many times you iteratively refine embeddings with self- and cross-attention (ALPHA-Fold-style recycling).
        self.padding_value = padding_value
        self.embed_dimension = embed_dimension

        self.logit_scale = nn.Parameter(torch.tensor(math.log(1/0.07)))  # ~CLIP init

        self.transformerencoder =  nn.TransformerEncoderLayer(
            d_model=self.embed_dimension,
            nhead=8,
            dropout=0.1,
            batch_first=True,
            dim_feedforward=self.embed_dimension
            )
 
        self.norm = nn.LayerNorm(self.embed_dimension)  # For residual additions

        self.cross_attn = nn.MultiheadAttention(
            embed_dim=self.embed_dimension,
            num_heads=8,
            dropout=0.1,
            batch_first=True
        )

        self.prot_embedder = nn.Sequential(
            nn.Linear(self.embed_dimension, 640),
            nn.ReLU(),
            nn.Linear(640, 320),
        )
        
    def forward(self, pep_input, prot_input, label=None, pep_int_mask=None, prot_int_mask=None, int_prob=None, mem_save=True): # , pep_tokens, prot_tokens

        pep_mask = create_key_padding_mask(embeddings=pep_input, padding_value=self.padding_value)
        prot_mask = create_key_padding_mask(embeddings=prot_input, padding_value=self.padding_value)
 
        # Initialize residual states
        pep_emb = pep_input.clone()
        prot_emb = prot_input.clone()
 
        for _ in range(self.num_recycles):

            # Transformer encoding with residual
            pep_trans = self.transformerencoder(self.norm(pep_emb), src_key_padding_mask=pep_mask)
            prot_trans = self.transformerencoder(self.norm(prot_emb), src_key_padding_mask=prot_mask)

            # Cross-attention with residual
            pep_cross, _ = self.cross_attn(query=self.norm(pep_trans), key=self.norm(prot_trans), value=self.norm(prot_trans), key_padding_mask=prot_mask)
            prot_cross, _ = self.cross_attn(query=self.norm(prot_trans), key=self.norm(pep_trans), value=self.norm(pep_trans), key_padding_mask=pep_mask)
            
            # Additive update with residual connection
            pep_emb = pep_emb + pep_cross  
            prot_emb = prot_emb + prot_cross

        pep_seq_coding = create_mean_of_non_masked(pep_emb, pep_mask)
        prot_seq_coding = create_mean_of_non_masked(prot_emb, prot_mask)
        
        # Use self-attention outputs for embeddings
        pep_seq_coding = F.normalize(self.prot_embedder(pep_seq_coding))
        prot_seq_coding = F.normalize(self.prot_embedder(prot_seq_coding))
 
        if mem_save:
            torch.cuda.empty_cache()
        
        scale = torch.exp(self.logit_scale).clamp(max=100.0)
        logits = scale * (pep_seq_coding * prot_seq_coding).sum(dim=-1)
        
        return logits

    def training_step(self, batch, device):
        embedding_pep, embedding_prot = batch
        embedding_pep, embedding_prot = embedding_pep.to(device), embedding_prot.to(device)
        
        positive_logits = self.forward(embedding_pep, embedding_prot)
        
        # Negative indexes
        rows, cols = torch.triu_indices(embedding_prot.size(0), embedding_prot.size(0), offset=1)         
        
        negative_logits = self(embedding_pep[rows,:,:], 
                          embedding_prot[cols,:,:], 
                          int_prob=0.0)

        # loss of predicting partner using peptide
        positive_loss = F.binary_cross_entropy_with_logits(positive_logits, torch.ones_like(positive_logits).to(device))
 
        # loss of predicting peptide using partner
        negative_loss =  F.binary_cross_entropy_with_logits(negative_logits, torch.zeros_like(negative_logits).to(device))
        
        loss = (positive_loss + negative_loss) / 2
 
        # del partner_prediction_loss, peptide_prediction_loss, embedding_pep, embedding_prot
        torch.cuda.empty_cache()
        return loss
    
    def validation_step(self, batch, device):
        # Predict on random batches of training batch size
        embedding_pep, embedding_prot = batch
        embedding_pep, embedding_prot = embedding_pep.to(device), embedding_prot.to(device)
        
        with torch.no_grad():

            positive_logits = self(embedding_pep, embedding_prot)
            
            # loss of predicting partner using peptide
            positive_loss = F.binary_cross_entropy_with_logits(positive_logits, torch.ones_like(positive_logits).to(device))
            
            # Negaive indexes
            rows, cols = torch.triu_indices(embedding_prot.size(0), embedding_prot.size(0), offset=1)
            
            negative_logits = self(embedding_pep[rows,:,:], embedding_prot[cols,:,:], int_prob=0.0)
    
            negative_loss =  F.binary_cross_entropy_with_logits(negative_logits, torch.zeros_like(negative_logits).to(device))

            loss = (positive_loss + negative_loss) / 2
           
            logit_matrix = torch.zeros((embedding_pep.size(0),embedding_pep.size(0)),device=self.device)
            logit_matrix[rows, cols] = negative_logits
            logit_matrix[cols, rows] = negative_logits
            
            # Fill diagonal with positive scores
            diag_indices = torch.arange(embedding_pep.size(0), device=self.device)
            logit_matrix[diag_indices, diag_indices] = positive_logits.squeeze()

            labels = torch.arange(embedding_prot.size(0)).to(self.device)
            peptide_predictions = logit_matrix.argmax(dim=0)
            peptide_ranks = logit_matrix.argsort(dim=0).diag() + 1
            peptide_mrr = (peptide_ranks).float().pow(-1).mean()
            
            # partner_accuracy = partner_predictions.eq(labels).float().mean()
            peptide_accuracy = peptide_predictions.eq(labels).float().mean()
    
            k = 3
            peptide_topk_accuracy = torch.any((logit_matrix.topk(k, dim=0).indices - labels.reshape(1, -1)) == 0, dim=0).sum() / logit_matrix.shape[0]
    
            del logit_matrix,positive_logits,negative_logits,embedding_pep,embedding_prot

            return loss, peptide_accuracy, peptide_topk_accuracy

    def calculate_logit_matrix(self,embedding_pep,embedding_prot):
        rows, cols = torch.triu_indices(embedding_pep.size(0), embedding_pep.size(0), offset=1)
        
        positive_logits = self(embedding_pep, embedding_prot)
        negative_logits = self(embedding_pep[rows,:,:], embedding_prot[cols,:,:], int_prob=0.0)
        
        logit_matrix = torch.zeros((embedding_pep.size(0),embedding_pep.size(0)),device=self.device)
        logit_matrix[rows, cols] = negative_logits
        logit_matrix[cols, rows] = negative_logits
        
        diag_indices = torch.arange(embedding_pep.size(0), device=self.device)
        logit_matrix[diag_indices, diag_indices] = positive_logits.squeeze()
        
        return logit_matrix

In [9]:
## Output path
trained_model_dir = "/zhome/c9/0/203261/DBL046_PP_osaul/DBL046_PP_osaul/tmp/ona_drafts"

## Embeddings paths
binders_embeddings = "/work3/s232958/data/PPint_DB/binders_embeddings_esm2"
targets_embeddings = "/work3/s232958/data/PPint_DB/targets_embeddings_esm2"

# ## Training variables
runID = uuid.uuid4()

def print_mem_consumption():
    # 1. Total memory available on the GPU (device 0)
    t = torch.cuda.get_device_properties(0).total_memory
    # 2. How much memory PyTorch has *reserved* from CUDA
    r = torch.cuda.memory_reserved(0)
    # 3. How much of that reserved memory is actually *used* by tensors
    a = torch.cuda.memory_allocated(0)
    # 4. Reserved but not currently allocated (so “free inside PyTorch’s pool”)
    f = r - a

    print("Total memory: ", t/1e9)      # total VRAM in GB
    print("Reserved memory: ", r/1e9)   # PyTorch’s reserved pool in GB
    print("Allocated memory: ", a//1e9) # actually in use (integer division)
    print("Free memory: ", f/1e9)       # slack in the reserved pool in GB
print_mem_consumption()

Total memory:  34.072559616
Reserved memory:  0.0
Allocated memory:  0.0
Free memory:  0.0


### Loading PPint dataframe

In [10]:
PPint_interaactions_df = pd.read_csv("/work3/s232958/data/PPint_DB/PPint_sample_muClustering.csv")
# PPint_interaactions_df = pd.read_csv("/zhome/c9/0/203261/DBL046_PP_osaul/DBL046_PP_osaul/tmp/data/PPint_DB/PPint_sample.csv")
PPint_interaactions_df["seq1_len"] = [len(seq) for seq in PPint_interaactions_df["seq1"].tolist()]
PPint_interaactions_df["seq2_len"] = [len(seq) for seq in PPint_interaactions_df["seq2"].tolist()]
PPint_interaactions_df["target_binder_id"] = PPint_interaactions_df["ID1"]+"_"+PPint_interaactions_df["ID2"]
PPint_interaactions_df = PPint_interaactions_df.rename(columns={
    "seq1": "target_seq",
    "seq2": "binder_seq",
    "ID1": "targetID",
    "ID2": "binderID",
    "seq1_len" : "seq_target_len",
    "seq2_len" : "seq_binder_len"
})
# PPint_interaactions_df["index"] = [i for i in PPint_interaactions_df["target_binder_id"].tolist()]
PPint_interaactions_df

,interface_id,target_seq,binder_seq,targetID,binderID,dimer,seq_target_len,seq_binder_len,target_binder_id,cluster,seq_target_len,seq_binder_len
0,1JEB_2,SLTKTERTIIVSMWAKISTQADTIGTETLERLFLSHPQTKTYFPHF...,VHLTDAEKAAVSGLWGKVNADEVGGEALGRLLVVYPWTQRYFDSFG...,1JEB_2_C,1JEB_2_D,False,141,146,1JEB_2_C_1JEB_2_D,852,141,146
1,7B12_23,TTTLAFKFQHGVIAAVDSRASAGSYISALRVNKVIEINPYLLGTMS...,DRGVNTFSPEGRLFQVEYAIEAIKLGSTAIGIQTSEGVCLAVEKRI...,7B12_23_Z,7B12_23_s,False,203,230,7B12_23_Z_7B12_23_s,523,203,230
2,6VCD_1,ITHLPPEVMLSIFSYLNPQELCRCSQVSMKWSQLTKTGSLWKHLYP...,PSIKLQSSDGEIFEVDVEIAKQSVTIKTMLEDLGDPVPLPNVNAAI...,6VCD_1_B,6VCD_1_C,False,255,135,6VCD_1_B_6VCD_1_C,2294,255,135
3,2OKG_0,NAKDVLGLTLLEKTLKERLNLKDAIIVSGDSDQSPWVKKEGRAAVA...,AKDVLGLTLLEKTLKERLNLKDAIIVSGDSDQSPWVKKEGRAAVAC...,2OKG_0_A,2OKG_0_B,True,241,243,2OKG_0_A_2OKG_0_B,8029,241,243
4,3MBX_0,DIVMSQSPSSLAVSVGEKVTMSCKSSQSLLYNNNQKNYLAWYQQKP...,VTLKESGPGILQPSQTLSLTCSFSGFSLSTYGMGVGWIRQPSGKGL...,3MBX_0_L,3MBX_0_H,False,220,229,3MBX_0_L_3MBX_0_H,82,220,229
...,...,...,...,...,...,...,...,...,...,...,...,...
2467,3HR4_0,REIPLKVLVKAVLFACMLMRKTMASRVRVTILFATETGKSEALAWD...,QLTEEQIAEFKEAFSLFDKDGDGTITTKELGTVMRSLGQNPTEAEL...,3HR4_0_A,3HR4_0_B,False,189,145,3HR4_0_A_3HR4_0_B,218,189,145
2468,6D3M_0,NKYRFIDVQPLTGVLGAEITGVDLREPLDDSTWNEILDAFHTYQVI...,KYRFIDVQPLTGVLGAEITGVDLREPLDDSTWNEILDAFHTYQVIY...,6D3M_0_A,6D3M_0_B,True,286,285,6D3M_0_A_6D3M_0_B,630,286,285
2469,4LUB_0,TMKIAYLGPSGSFTHNVALHAFPAADLLPFENITEVIKAYESKQVC...,TMKIAYLGPSGSFTHNVALHAFPAADLLPFENITEVIKAYESKQVC...,4LUB_0_A,4LUB_0_B,True,188,190,4LUB_0_A_4LUB_0_B,9344,188,190
2470,4MN4_2,ERDEVGARKNAVDEEIERLSQPGDQRLNALAERFGGVLLSEIYDDV...,EPVTIVLSQGWVRSAKGHDIDAPGLNYKAGDSFKAAVKGKSNQPVV...,4MN4_2_D,4MN4_2_B,False,154,236,4MN4_2_D_4MN4_2_B,1366,154,236


In [11]:
clusters_counts_Dict = dict(PPint_interaactions_df.cluster.value_counts())
clusters_counts = list(clusters_counts_Dict.items())

random.seed(0)
random.shuffle(clusters_counts)

counter = 0
ceil = round(len(PPint_interaactions_df) * 0.2)
val_clusters = []

for (clust, num) in clusters_counts:
    
    if num + counter < ceil:
        counter += num
        val_clusters.append(clust)

    elif num + counter > ceil:
        continue

    elif num + counter == ceil:
        counter += num
        val_clusters.append(clust)
        break

counter = 0
for c in val_clusters:
    counter += clusters_counts_Dict[c]
print(counter)
print(ceil)

494
494


In [12]:
Df_val = PPint_interaactions_df[PPint_interaactions_df.cluster.isin(val_clusters)]
Df_train = PPint_interaactions_df.drop(Df_val.index)
Df_val

,interface_id,target_seq,binder_seq,targetID,binderID,dimer,seq_target_len,seq_binder_len,target_binder_id,cluster,seq_target_len,seq_binder_len
8,6D40_0,PSFDCGKPQVEPKKCPVVGGCVAHPHSWPWQVSLRTRFGMHFCGGT...,GRCYKSIPPICFPD,6D40_0_A,6D40_0_C,False,246,14,6D40_0_A_6D40_0_C,59,246,14
10,5N11_0,YNPPTNVVSHVNGDWFLFGDSRSDCNHIGNINPHNYSYMDLNPALC...,YNPPTNVVSHVNGDWFLFGDSRSDCNHIGNINPHNYSYMDLNPALC...,5N11_0_A,5N11_0_B,True,342,348,5N11_0_A_5N11_0_B,11609,342,348
12,6NB5_0,EVQLLESGGGLVKPGGSLRLSCEASGLTFSNVWMSWVRQAPGKGLE...,ALTQPASVSGSPGQSITISCTGTSSDVGTYDLVSWYQQHPGKSPKL...,6NB5_0_H,6NB5_0_L,False,220,210,6NB5_0_H_6NB5_0_L,59,220,210
19,6CK9_1,QGQLVQSGATTTKPGSSVKISCKTSGYRFNFYHINWIRQTAGRGPE...,VLTQSASVSGSLGQSVTISCTGPNSVCCSHKSISWYQWPPGRAPTL...,6CK9_1_D,6CK9_1_E,False,128,100,6CK9_1_D_6CK9_1_E,59,128,100
21,2Z0G_1,MKKIEAIIKPFKLDEVKDALVEIGIGGMTVTEVKGFDFLPKVKIEV...,MKKIEAIIKPFKLDEVKDALVEIGIGGMTVTEVKGFDFLPKVKIEV...,2Z0G_1_B,2Z0G_1_C,True,95,95,2Z0G_1_B_2Z0G_1_C,4689,95,95
...,...,...,...,...,...,...,...,...,...,...,...,...
2426,7S8T_5,KMTDTELARSIRLNIEAELDAINLYAAHIDATDNEDAKAILQHVMD...,KMTDTELARSIRLNIEAELDAINLYAAHIDATDNEDAKAILQHVMD...,7S8T_5_F,7S8T_5_G,True,80,80,7S8T_5_F_7S8T_5_G,10979,80,80
2440,3ZPN_0,AMAEIQFIRGINEEVVPDVRLTRARDGSSGQAMFYFDNPKIVQEGN...,AMAEIQFIRGINEEVVPDVRLTRARDGSSGQAMFYFDNPKIVQEGN...,3ZPN_0_A,3ZPN_0_B,True,108,107,3ZPN_0_A_3ZPN_0_B,6348,108,107
2446,3P19_1,KKLVVITGASSGIGEAIARRFSEEGHPLLLLARRVERLKALNLPNT...,KKLVVITGASSGIGEAIARRFSEEGHPLLLLARRVERLKALNLPNT...,3P19_1_B,3P19_1_D,True,218,238,3P19_1_B_3P19_1_D,8608,218,238
2447,7SZ3_0,KEICLDHLYRGCQFHLPYRWQLFILPTWMDFQDMEYIERAYCDPQI...,KEICLDHLYRGCQHFHLPYRWQLFILPTWMDFQDMEYIERAYCDPQ...,7SZ3_0_A,7SZ3_0_B,True,175,179,7SZ3_0_A_7SZ3_0_B,7430,175,179


In [15]:
### checking shape (size) of the encodings for later
fname = os.listdir("/work3/s232958/data/PPint_DB/binders_embeddings_esm2")[153]
np.load(os.path.join("/work3/s232958/data/PPint_DB/binders_embeddings_esm2", fname)).shape[1]

1280

In [16]:
"6VCD_1_B_6VCD_1_C".split("_")

['6VCD', '1', 'B', '6VCD', '1', 'C']

In [22]:
class CLIP_PPint_analysis_dataset(Dataset):
    def __init__(self, dframe, tpath, bpath, embedding_dim=1280, padding_value=-5000.0):
        super().__init__()

        self.dframe = dframe.copy()
        self.max_tlen = self.dframe["seq_target_len"].max()[0]
        self.max_blen = self.dframe["seq_binder_len"].max()[0]
        self.encoding_tpath = tpath
        self.encoding_bpath = bpath
        self.dframe.set_index("target_binder_id", inplace=True)
        self.accessions = self.dframe.index.astype(str).tolist()
        self.name_to_row = {name: i for i, name in enumerate(self.accessions)} # 1YXO_0_1YXO_0 : 1, ...
        self.samples = []

        iterator = tqdm(self.accessions, position=0, total=len(self.accessions), desc="#Loading ESM2 embeddings")

        for accession in iterator:
            parts = accession.split("_")
            
            if len(parts) < 6:
                raise ValueError(
                    f"Expected target_binder_id to have at least 4 underscore-separated parts, got {accession}")

            target_id = parts[0] + "_" + parts[1] + "_" + parts[2]
            binder_id = parts[3] + "_" + parts[4] + "_" + parts[5]

            tname = f"{target_id}"
            bname = f"{binder_id}"

            tnpy_path = os.path.join(self.encoding_tpath, f"{tname}.npy")
            bnpy_path = os.path.join(self.encoding_bpath, f"{bname}.npy")

            if not os.path.exists(tnpy_path):
                raise FileNotFoundError(f"Missing target embedding file: {tnpy_path}")
            if not os.path.exists(bnpy_path):
                raise FileNotFoundError(f"Missing binder embedding file: {bnpy_path}")

            tembd = np.load(tnpy_path)
            if tembd.shape[0] < self.max_tlen:
                t_pad_len = self.max_tlen - tembd.shape[0]
                t_pad = np.full((t_pad_len, tembd.shape[1]), padding_value, dtype=tembd.dtype)
                t_final = np.concatenate([tembd, t_pad], axis=0)
            else:
                t_final = tembd[: self.max_tlen]

            bembd = np.load(bnpy_path)
            if bembd.shape[0] < self.max_blen:
                b_pad_len = self.max_blen - bembd.shape[0]
                b_pad = np.full((b_pad_len, bembd.shape[1]), padding_value, dtype=bembd.dtype)
                b_final = np.concatenate([bembd, b_pad], axis=0)
            else:
                b_final = bembd[: self.max_blen]

            # label = 1
            # self.samples.append((t_final, b_final, label))
            self.samples.append((t_final, b_final))


    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        t_arr, b_arr = self.samples[idx]
        # t_arr, b_arr, y = self.samples[idx]
        t_tensor = torch.tensor(t_arr, dtype=torch.float32)
        b_tensor = torch.tensor(b_arr, dtype=torch.float32)
        # y_tensor = torch.tensor(y, dtype=torch.float32)
        return  b_tensor, t_tensor #, y_tensor

    def _get_by_name(self, name):
        
        # only one index parsed
        if isinstance(name, str):
            idx = self.name_to_row[name]
            binder_tensor, target_tensor = self.__getitem__(idx)
            
            return binder_tensor, target_tensor

        # parsing list of indeces
        binder_list = []
        target_list = []
        for n in name:
            idx = self.name_to_row[n]
            b_tensor, t_tensor = self.__getitem__(idx)
            binder_list.append(b_tensor)
            target_list.append(t_tensor)

        # stack along batch dim = 0
        binder_batch = torch.stack(binder_list, dim=0)  # [B, max_blen, embedding_dim]
        target_batch = torch.stack(target_list, dim=0)  # [B, max_tlen, embedding_dim]

        return binder_batch, target_batch

targets_path ="/work3/s232958/data/PPint_DB/targets_embeddings_esm2"
binders_path ="/work3/s232958/data/PPint_DB/binders_embeddings_esm2"

training_Dataset = CLIP_PPint_analysis_dataset(Df_train, tpath=targets_path, bpath=binders_path, embedding_dim=1280)
validation_Dataset = CLIP_PPint_analysis_dataset(Df_val, tpath=targets_path, bpath=binders_path, embedding_dim=1280)

/work3/s232958/tmp/ipykernel_3855838/964388305.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  self.max_tlen = self.dframe["seq_target_len"].max()[0]
/work3/s232958/tmp/ipykernel_3855838/964388305.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  self.max_blen = self.dframe["seq_binder_len"].max()[0]
#Loading ESM2 embeddings: 100%|████████████████████████████████████████████████████████████████████████| 1978/1978 [00:15<00:00, 124.36it/s]
/work3/s232958/tmp/ipykernel_3855838/964388305.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always 

In [23]:
indices_non_dimers_val = Df_val[~Df_val["dimer"]].index.tolist()
indices_non_dimers_val[:5]

accessions = [Df_val.loc[index].target_binder_id for index in indices_non_dimers_val]
validation_Dataset._get_by_name(accessions[:5])

(tensor([[[ 5.8141e-02,  4.8300e-02, -8.6795e-02,  ..., -3.4068e-01,
            3.0144e-01, -7.1343e-02],
          [ 1.4835e-01,  5.8364e-02, -1.2591e-03,  ...,  1.1050e-01,
           -6.0273e-02, -5.9261e-02],
          [ 1.4789e-01, -1.0866e-02,  1.8140e-03,  ..., -1.2451e-01,
           -1.0935e-02, -1.2974e-02],
          ...,
          [-5.0000e+03, -5.0000e+03, -5.0000e+03,  ..., -5.0000e+03,
           -5.0000e+03, -5.0000e+03],
          [-5.0000e+03, -5.0000e+03, -5.0000e+03,  ..., -5.0000e+03,
           -5.0000e+03, -5.0000e+03],
          [-5.0000e+03, -5.0000e+03, -5.0000e+03,  ..., -5.0000e+03,
           -5.0000e+03, -5.0000e+03]],
 
         [[ 9.4384e-02, -1.4001e-02,  6.3385e-03,  ..., -2.0922e-01,
            2.4640e-01, -5.6498e-03],
          [ 1.0727e-01,  1.0003e-01, -1.0205e-01,  ...,  2.3739e-01,
           -5.0126e-02, -9.3079e-02],
          [-2.5306e-02, -2.7954e-01,  1.5058e-01,  ..., -1.1037e-02,
            4.9918e-01,  8.1231e-03],
          ...,
    

### Train model from scratch with 10% of PPint dataset using old architecture (encodings only)

In [24]:
model = MiniCLIP_w_transformer_crossattn(embed_dimension=embedding_dimension, num_recycles=number_of_recycles)
model.to("cuda")

MiniCLIP_w_transformer_crossattn(
  (transformerencoder): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=1280, out_features=1280, bias=True)
    )
    (linear1): Linear(in_features=1280, out_features=1280, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (linear2): Linear(in_features=1280, out_features=1280, bias=True)
    (norm1): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.1, inplace=False)
    (dropout2): Dropout(p=0.1, inplace=False)
  )
  (norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
  (cross_attn): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=1280, out_features=1280, bias=True)
  )
  (prot_embedder): Sequential(
    (0): Linear(in_features=1280, out_features=640, bias=True)
    (1): ReLU()
    (2): Linear(in_features=640, out_features=32

### Trianing loop

In [25]:
def batch(iterable, n=1):
    """Takes any indexable iterable (e.g., a list of observation IDs) and yields contiguous slices of length n."""
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

class TrainWrapper():

    def __init__(self, 
                 model, 
                 training_loader, 
                 validation_loader,
                 test_df,
                 test_dataset, 
                 optimizer, 
                 EPOCHS, 
                 runID, 
                 device, 
                 test_indexes_for_auROC = None,
                 auROC_batch_size=10, 
                 model_save_steps=False, 
                 model_save_path=False, 
                 v=False, 
                 wandb_tracker=False):
        
        self.model = model 
        self.training_loader = training_loader
        self.validation_loader = validation_loader
        self.EPOCHS = EPOCHS
        self.wandb_tracker = wandb_tracker
        self.model_save_steps = model_save_steps # if truthy (e.g., 1, 5), save a checkpoint every N epochs.
        self.verbose = v
        self.best_vloss = 1_000_000
        self.optimizer = optimizer
        self.runID = runID
        self.trained_model_dir = model_save_path
        self.print_frequency_loss = 1
        self.device = device
        self.test_indexes_for_auROC = test_indexes_for_auROC
        self.auROC_batch_size = auROC_batch_size # which observation indices to use when computing auROC/auPR periodically.
        self.test_dataset = test_dataset
        self.test_df = test_df

    def train_one_epoch(self):

        self.model.train() 
        running_loss = 0 

        for batch in tqdm(self.training_loader, total=len(self.training_loader), desc="Running through epoch"):
            
            if batch[0].size(0) == 1: 
                continue
            
            self.optimizer.zero_grad() 
            loss = self.model.training_step(batch, self.device) 
            loss.backward() 
            self.optimizer.step() 

            torch.cuda.empty_cache()
            running_loss += loss.item()
            
        return running_loss / len(self.training_loader)
    
    def calc_auroc_aupr_on_indexes(self, model, dataset, test_df, nondimer_indexes, batch_size = 10):

        self.model.eval()
        all_TP_scores, all_FP_scores = [], []
        accessions = [test_df.loc[index].target_binder_id for index in nondimer_indexes]
        batches_local = batch(accessions, n=batch_size)
        
        with torch.no_grad():
            for index_batch in tqdm(batches_local, total=int(len(accessions)/batch_size), desc="Calculating AUC"):

                # Loading the data into the GPU based on the index batch
                binder_emb, target_emb = dataset._get_by_name(index_batch)
                binder_emb, target_emb = binder_emb.to(self.device), target_emb.to(self.device)

                logit_matrix = self.model.calculate_logit_matrix(binder_emb, target_emb)
                
                TP_scores = logit_matrix.diag().detach().cpu().tolist()
                all_TP_scores += TP_scores
                
                # Get FP scores from upper triangle (excluding diagonal)
                n = logit_matrix.size(0)
                rows, cols = torch.triu_indices(n, n, offset=1)
                FP_scores = logit_matrix[rows, cols].detach().cpu().tolist()
                all_FP_scores += FP_scores
            
        # Calculate scores and labels
        all_score_predictions = np.array(all_TP_scores + all_FP_scores)
        all_labels = np.array([1]*len(all_TP_scores) + [0]*len(all_FP_scores))
                
        # Calculate ROC curve metrics
        fpr, tpr, thresholds = metrics.roc_curve(all_labels, all_score_predictions)
        auroc = metrics.roc_auc_score(all_labels, all_score_predictions)
        aupr = metrics.average_precision_score(all_labels, all_score_predictions)
        
        return auroc, aupr, all_TP_scores, all_FP_scores


    def validate (self, dataloader, indexes_for_auc=False, auROC_dataset=False, auROC_df = False):
        self.model.eval()
        running_loss = 0.0
        running_accuracy = 0.0
        running_topk_accuracy = 0.0

        with torch.no_grad():
            for batch in tqdm(dataloader, total=len(dataloader), desc="First Validation run"):
                if batch[0].size(0) == 1: # We can't make negatives on a batch of 1
                    continue
                loss, partner_accuracy, peptide_topk_accuracy = self.model.validation_step(batch, self.device)
                running_loss += loss.item()
                running_accuracy += partner_accuracy.item()
                running_topk_accuracy += peptide_topk_accuracy.item()
                
            val_loss = running_loss / len(dataloader)
            val_accuracy = running_accuracy / len(dataloader)
            val_topk_accuracy = running_topk_accuracy / len(dataloader)

            if indexes_for_auc:
                # Calculating auc-scores
                non_dimer_auc, non_dimer_aupr, ___, ___  = self.calc_auroc_aupr_on_indexes(model=self.model, 
                                                                                           dataset=auROC_dataset,
                                                                                           test_df=auROC_df,
                                                                                           nondimer_indexes=indexes_for_auc,
                                                                                           batch_size=self.auROC_batch_size)
                
                return val_loss, val_accuracy, val_topk_accuracy, non_dimer_auc, non_dimer_aupr
            
            else:
                return val_loss, val_accuracy, val_topk_accuracy

    def train_model(self):

        if self.verbose:
            print(f"Training model {str(self.runID)}")
        
        # --- initial validation before training
        if self.test_indexes_for_auROC:
            val_loss_before, val_accuracy_before, val_topk_accuracy_before, val_nondimer_auc_before, val_nondimer_aupr_before = self.validate(
                dataloader=self.validation_loader,
                indexes_for_auc=self.test_indexes_for_auROC,
                auROC_dataset=self.test_dataset,
                auROC_df=self.test_df
            )
        else:
            val_loss_before, val_accuracy_before, val_topk_accuracy_before = self.validate(
                dataloader=self.validation_loader,
                indexes_for_auc=self.test_indexes_for_auROC,
                auROC_dataset=self.test_dataset
            )
            val_nondimer_auc_before = None
            val_nondimer_aupr_before = None
        
        if self.verbose: 
            if val_nondimer_auc_before is not None:
                print(f'Before training - Val CLIP-loss {round(val_loss_before,4)}',
                      f'Accuracy: {round(val_accuracy_before,4)}',
                      f'Top-K accuracy : {round(val_topk_accuracy_before,4)}',
                      f'auc: {round(val_nondimer_auc_before,3)}')
            else:
                print(f'Before training - Val CLIP-loss {round(val_loss_before,4)}',
                      f'Accuracy: {round(val_accuracy_before,4)}',
                      f'Top-K accuracy : {round(val_topk_accuracy_before,4)}')

        if self.wandb_tracker:
            metrics_to_log = {
                "Val-loss": val_loss_before,
                "Val-acc": val_accuracy_before,
                "Val-TOPK-acc": val_topk_accuracy_before,
            }
            if val_nondimer_auc_before is not None:
                metrics_to_log["Val non-dimer auc"] = val_nondimer_auc_before
                metrics_to_log["Val non-dimer auPR"] = val_nondimer_aupr_before
            self.wandb_tracker.log(metrics_to_log)
        
        # --- training loop
        for epoch in tqdm(range(1, self.EPOCHS + 1), total=self.EPOCHS, desc="Epochs"):
            
            train_loss = self.train_one_epoch()
            
            # validation after epoch
            if self.test_indexes_for_auROC:
                val_loss, val_accuracy, val_topk_accuracy, non_dimer_auc, non_dimer_aupr = self.validate(
                    dataloader=self.validation_loader,
                    indexes_for_auc=self.test_indexes_for_auROC,
                    auROC_dataset=self.test_dataset,
                    auROC_df=self.test_df
                )
            else:
                val_loss, val_accuracy, val_topk_accuracy = self.validate(
                    dataloader=self.validation_loader,
                    indexes_for_auc=self.test_indexes_for_auROC,
                    auROC_dataset=self.test_dataset,
                    auROC_df=self.test_df
                )
                non_dimer_auc = None
                non_dimer_aupr = None
            
            # checkpoint save
            if self.model_save_steps:
                if epoch % self.model_save_steps == 0:
                    check_point_folder = os.path.join(
                        self.trained_model_dir,
                        f"{str(self.runID)}_checkpoint_{str(epoch)}"
                    )
                    if self.verbose:
                        print("Saving model to:", check_point_folder)
                    
                    if not os.path.exists(check_point_folder):
                        os.makedirs(check_point_folder)

                    checkpoint_path = os.path.join(
                        check_point_folder,
                        f"{str(self.runID)}_checkpoint_epoch_{str(epoch)}.pth"
                    )
                    torch.save(
                        {
                            'epoch': epoch,
                            'model_state_dict': self.model.state_dict(),
                            'optimizer_state_dict': self.optimizer.state_dict(),
                            'val_loss': val_loss
                        }, 
                        checkpoint_path
                    )
            
            # console logging
            if self.verbose:
                if epoch % self.print_frequency_loss == 0:
                    if non_dimer_auc is not None:
                        print(f'EPOCH {epoch} -  Val loss {round(val_loss,4)}',
                              f'Accuracy: {round(val_accuracy,4)}',
                              f'Top-K accuracy: {round(val_topk_accuracy,4)}',
                              f'Val-Auc:{round(non_dimer_auc,3)}', 
                              f'Val-auPR:{round(non_dimer_aupr,3)}')
                    else:
                        print(f'EPOCH {epoch} -  Val loss {round(val_loss,4)}',
                              f'Accuracy: {round(val_accuracy,4)}',
                              f'Top-K accuracy: {round(val_topk_accuracy,4)}')
            
            # wandb logging
            if self.wandb_tracker:
                metrics_to_log_epoch = {
                    "Epoch": epoch,
                    "Train-loss": train_loss,
                    "Val-loss": val_loss,
                    "Val-acc": val_accuracy,
                    "Val-TOPK-acc": val_topk_accuracy,
                }
                if non_dimer_auc is not None:
                    metrics_to_log_epoch["Val non-dimer auc"] = non_dimer_auc
                    metrics_to_log_epoch["Val non-dimer auPR"] = non_dimer_aupr
                self.wandb_tracker.log(metrics_to_log_epoch)

        if self.wandb_tracker:
            self.wandb_tracker.finish()

In [26]:
batch_size = 10
learning_rate = 2e-5
EPOCHS = 5
g = torch.Generator().manual_seed(SEED)

# login once (env var preferred)
if use_wandb:
    import wandb
    wandb.login()

optimizer = AdamW(model.parameters(), lr=learning_rate)
train_dataloader = DataLoader(training_Dataset, batch_size=10)
val_dataloader = DataLoader(validation_Dataset, batch_size=20, shuffle=False, drop_last = False)

# accelerator
accelerator = Accelerator()
device = accelerator.device
model, optimizer, train_dataloader, val_dataloader = accelerator.prepare(model, optimizer, train_dataloader, val_dataloader)

# wandb
if use_wandb:
    run = wandb.init(
        project="PPint_retrain_w_10percent_ofdata",
        name=f"PPint_retrain_afterMMSeq0.4",
        config={"learning_rate": learning_rate, "batch_size": batch_size, "epochs": EPOCHS,
                "architecture": "MiniCLIP_w_transformer_crossattn", "dataset": "Meta analysis"},
    )
    wandb.watch(accelerator.unwrap_model(model), log="all", log_freq=100)
else:
    run = None

# train
training_wrapper = TrainWrapper(model=model, 
                                training_loader = train_dataloader, 
                                validation_loader = val_dataloader, 
                                test_dataset = validation_Dataset,
                                test_df = Df_val,
                                optimizer = optimizer, 
                                EPOCHS = EPOCHS,
                                runID = runID, 
                                device = device, 
                                test_indexes_for_auROC = indices_non_dimers_val, 
                                model_save_steps = model_save_steps,
                                model_save_path = trained_model_dir, 
                                v = True, 
                                wandb_tracker = wandb
                                )

training_wrapper.train_model()

Training model db449825-19c3-450e-a7a0-c463ba6c9030


First Validation run: 100%|█████████████████████████████████████████████████████████████████████████████████| 25/25 [00:38<00:00,  1.55s/it]
Calculating AUC: 20it [00:08,  2.38it/s]                                                                                                    


Before training - Val CLIP-loss 6.1371 Accuracy: 0.7 Top-K accuracy : 0.8 auc: 0.658


Running through epoch: 100%|██████████████████████████████████████████████████████████████████████████████| 198/198 [04:33<00:00,  1.38s/it]

First Validation run: 100%|█████████████████████████████████████████████████████████████████████████████████| 25/25 [00:39<00:00,  1.57s/it]

Calculating AUC:  95%|█████████████████████████████████████████████████████████████████████████████████▍    | 18/19 [00:07<00:00,  2.39it/s]
Calculating AUC: 20it [00:08,  2.49it/s]                                                                                                    


Saving model to: /zhome/c9/0/203261/DBL046_PP_osaul/DBL046_PP_osaul/tmp/ona_drafts/db449825-19c3-450e-a7a0-c463ba6c9030_checkpoint_1


...


Running through epoch: 100%|██████████████████████████████████████████████████████████████████████████████| 198/198 [04:33<00:00,  1.38s/it]

First Validation run: 100%|█████████████████████████████████████████████████████████████████████████████████| 25/25 [00:38<00:00,  1.55s/it]

Calculating AUC:  95%|█████████████████████████████████████████████████████████████████████████████████▍    | 18/19 [00:07<00:00,  2.41it/s]
Calculating AUC: 20it [00:08,  2.41it/s]                                                                                                    


Saving model to: /zhome/c9/0/203261/DBL046_PP_osaul/DBL046_PP_osaul/tmp/ona_drafts/db449825-19c3-450e-a7a0-c463ba6c9030_checkpoint_5


Epochs: 100%|████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [26:57<00:00, 323.59s/it]

EPOCH 5 -  Val loss 0.308 Accuracy: 0.77 Top-K accuracy: 0.88 Val-Auc:0.785 Val-auPR:0.436


Epoch,▁▃▅▆█
Train-loss,█▃▂▁▁
Val non-dimer auPR,▁▁▃▅▆█
Val non-dimer auc,▁▆▇▇▇█
Val-TOPK-acc,▁▆▇█▇█
Val-acc,▁▃▅▇██
Val-loss,█▁▁▁▁▁
Epoch,5
Train-loss,0.09469
Val non-dimer auPR,0.43557
Val non-dimer auc,0.78488
